In [44]:
import numpy as np

'''
generate data set SYNSEP
a 9-class, 400-dimensional synthetic data set of size 10^6
'''
d = 400  # number of features
K = 9  # number of classes
n = 100000  # number of data
# generate the fixed 9 bit-vectors
V = np.zeros([d, K], int)
idx_support = np.zeros([120, 9], int)
Nsupport = np.zeros([9], int)
for i in range(K):
    idx_permuted = np.random.permutation(range(120))
    idx_support[:, i] = idx_permuted
    Nsupport[i] = np.random.randint(20, 41)
    V[idx_permuted[:Nsupport[i]], i] = np.ones([Nsupport[i]])


In [45]:
# generate the data set
X = np.zeros([d, n])
y = np.zeros([n])
for i in range(n):
    j = np.random.randint(0, 9)
    y[i] = j+1
    X[:, i] = V[:, j]
    idx_turnoff = np.random.permutation(idx_support[:Nsupport[j], j])[:5]
    idx_turnon = np.random.permutation(range(120, 400))[:20]
    X[idx_turnoff, i] = np.zeros([5])
    X[idx_turnon, i] = np.ones([20])

In [46]:
'''
Second Order Banditron Algorithm
'''
# initialization
a = 1  # regularization parameter
gamma = 0.01  # exploration parameter
T = n  # number of rounds
W = np.zeros([K, d])
A = a * np.ones([K*d])
theta = np.zeros([K*d])
E = np.eye(K)
cumulative_margin = 0
z = np.zeros([K*d])
g = np.zeros([K*d])
correct = 0
# SOBA algorithm
for t in range(T):
    Wx = W.dot(X[:, t])
    yhat = np.argmax(Wx) + 1
    p = (1-gamma) * E[:, yhat-1] + gamma / K * np.ones([K])
    rand_num = np.random.random()
    for i in range(K):
        if sum(p[:i+1]) >= rand_num:
            ytilde = i+1
            break
    nt = 0
    if ytilde == y[t]:
        correct += 1
        Wx_bar = np.zeros([K-1])
        Wx_bar[:ytilde-1] = Wx[:ytilde-1]
        Wx_bar[ytilde-1:] = Wx[ytilde:]
        ybar = np.argmax(Wx)+1
        g = 1 / p[ytilde-1] * np.kron(E[:, ybar-1]-E[:, ytilde-1], X[:, t])
        z = np.sqrt(p[ytilde-1]) * g
        m = (sum(W.reshape(-1, 1)*z.reshape(-1, 1))**2 + 2*sum(W.reshape(-1, 1)*g.reshape(-1, 1))) / (1 + sum(z*z/A))
        if cumulative_margin+m >= 0:
            nt = 1
        cumulative_margin += nt*m
    A += nt*z*z
    theta -= nt*g
    W = (theta/A).reshape(K, d)
print(correct/T)

0.88191